In [ ]:
import pandas as pd
import openai

df = pd.read_csv("final_images.csv")


In [ ]:
import openai

def get_average_size(object_name, model_version="gpt-4o"):
    API_KEY = "YOUR API KEY HERE"
    client = openai.OpenAI(api_key=API_KEY)

    prompt = f"What is the average size of a {object_name}? Give your final answer in the format number_length feet x number_width feet. So for example, final answer: 3.7 feet x 10.13 feet."
    
    response = client.chat.completions.create(
        model=model_version,
        messages=[
            {"role": "system", "content": "Provide average size in feet for a given object."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000,
        temperature=0,
        top_p=1
    )
    
    return response.choices[0].message.content.strip()

# Loop through dataframe and get average size
sizes = []
for obj in df["correct_object"]:
    avg_size = get_average_size(obj)
    sizes.append(avg_size)

df["average_size"] = sizes

In [ ]:
df["average_size"] = df["average_size"].str.replace("foot", "feet")

In [ ]:
df["average_size"] = df["average_size"].str.replace("N/A", "0.5")

In [ ]:
import re
def extract_most_precise_measurements(text):
    """
    Extracts the most precise 'feet x feet' format measurement from a given text.
    Prioritizes values that explicitly follow 'approximately' or are in the final stated dimensions.
    """
    # Look for 'approximately X feet x Y feet' explicitly
    pattern = r'approximately\s+(\d*\.?\d+)\s*feet.*?\bx\b.*?(\d*\.?\d+)\s*feet'
    
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return f"{match.group(1)} feet x {match.group(2)} feet"

    # Fallback: If 'approximately' is not present, try generic extraction
    fallback_pattern = r'(\d*\.?\d+)\s*feet\s*\(?.*?\)?\s*x\s*(\d*\.?\d+)\s*feet'
    fallback_match = re.findall(fallback_pattern, text, re.IGNORECASE)
    
    if fallback_match:
        return f"{fallback_match[-1][0]} feet x {fallback_match[-1][1]} feet"
    
    return None

df["extracted_size"] = df["average_size"].apply(extract_most_precise_measurements)


In [ ]:
def calculate_total_size(extracted_size):
    """
    Calculates the total size by multiplying the two extracted dimensions in feet.
    """
    try:
        # Extract numerical values from the 'feet x feet' format
        dimensions = re.findall(r'(\d*\.?\d+)', extracted_size)
        if len(dimensions) == 2:
            width, height = map(float, dimensions)
            return width * height  # Compute total size as area in square feet
    except:
        return None  # Return None if extraction fails

# Apply the function to create the "total_size" column
df["total_size"] = df["extracted_size"].apply(calculate_total_size)
df["total_size"]

In [ ]:
df = df[~df["total_size"].isna()]

In [ ]:
df.to_csv("final_images_extracted_sizes.csv", index=False)

In [ ]:
excluded_objects = [
    "American egret",
    "cottage",
    "hotdog",
    "swab",
    "radio telescope",
    "cab",
    "carbonara",
    "three-toed sloth",
    "computer keyboard",
    "cupboard",
    "theater curtain",
    "dugong",
    "electric fan",
    "indigo bunting",
    "solar dish",
    "komondor",
    "limpkin",
    "mashed potato",
    "mortar",
    "padlock",
    "strainer",
    "rotisserie",
    "white stork",
    "yacht",
    "machete"
]
df = df[~df["correct_object"].isin(excluded_objects)]
len(df)

In [ ]:
import pandas as pd
import numpy as np

def find_closest_match(df, target_size, factor, exclude_object):
    """
    Finds the closest valid object in df with a total_size at least `factor` times larger/smaller, 
    excluding the given object.
    """
    df_filtered = df[df["correct_object"] != exclude_object].copy()  # Exclude itself
    df_filtered["size_ratio"] = df_filtered["total_size"] / target_size

    # Expand valid range (5x to 500x instead of a strict 100x)
    if factor > 1:  # Looking for a larger object
        df_filtered = df_filtered[df_filtered["size_ratio"] >= 10]  # Ensure at least 5x larger
    else:  # Looking for a smaller object
        df_filtered = df_filtered[df_filtered["size_ratio"] <= 0.1]  # Ensure at least 5x smaller

    if df_filtered.empty:
        return None, None

    # Select the closest match by absolute size ratio difference
    closest_match = df_filtered.iloc[(df_filtered["size_ratio"] - factor).abs().argsort()[:1]]
    return closest_match["correct_object"].values[0], closest_match["total_size"].values[0]

# Create new dataframe for object pairings with proper exclusions and valid scaling
expanded_rows = []

for _, row in df.iterrows():
    obj_name = row["correct_object"]
    obj_size = row["total_size"]

    # Find a larger object (5x-500x bigger), ensuring a valid match or None
    larger_obj, larger_size = find_closest_match(df, obj_size, 200, obj_name)

    # Find a smaller object (5x-500x smaller), ensuring a valid match or None
    smaller_obj, smaller_size = find_closest_match(df, obj_size, 1/200, obj_name)

    expanded_rows.append([obj_name, obj_size, larger_obj, larger_size, smaller_obj, smaller_size])


In [ ]:
df_pairings_final = pd.DataFrame(expanded_rows, columns=["correct_object", "total_size", "larger_object", "larger_size", "smaller_object", "smaller_size"])

df_combined = df.merge(df_pairings_final, on=["correct_object", "total_size"], how="left")

df_combined

In [ ]:
for idx, row in df_combined.iterrows():
    obj = row["correct_object"]
    larger = row["larger_object"]
    smaller = row["smaller_object"]

    if larger not in df_combined["correct_object"].values:
        print(f"[{obj}] Missing larger_object: {larger}")
    
    if smaller not in df_combined["correct_object"].values:
        print(f"[{obj}] Missing smaller_object: {smaller}")


In [ ]:
df_combined.to_csv("images_extracted_sizes_cleaned.csv", index=False)

In [ ]:
import pickle
import numpy as np

# Define the file paths for MASKS
color_path = "final_real_images_masks_color.pkl"
white_path = "final_real_images_masks_white.pkl"

# Load both dictionaries
with open(color_path, "rb") as f:
    color_data = pickle.load(f)

with open(white_path, "rb") as f:
    white_data = pickle.load(f)

# Combine the dictionaries
combined_data = {}

# Combine logic
for key in set(color_data.keys()).union(white_data.keys()):
    mask_color = color_data.get(key)
    mask_white = white_data.get(key)

    if mask_color is not None and mask_white is not None:
        # Example: combine masks with logical OR
        combined_data[key] = np.logical_or(mask_color, mask_white).astype(np.float32)
    elif mask_color is not None:
        combined_data[key] = mask_color
    else:
        combined_data[key] = mask_white


In [ ]:
color_data

In [ ]:
white_data

In [ ]:
import pandas as pd
mask_df = pd.DataFrame(list(combined_data.items()), columns=["corrupt_image_path", "mask"])


In [ ]:
def generate_counterfact_path(row):
    color_to_replace = row['image_path'].split('_')[1]
    new_path = row['image_path'].replace(color_to_replace, row['incorrect_answer'])
    new_path = new_path.replace('downloaded_images', 'downloaded_images_counterfact')
    return new_path

df["corrupt_image_path"] = df.apply(generate_counterfact_path, axis=1)
df["corrupt_image_path"]

In [ ]:
df = df.merge(mask_df, on="corrupt_image_path", how="left")

In [ ]:
from PIL import Image, ImageDraw
import numpy as np
from IPython.display import display
import random
import os

composite_dir = "composites"
os.makedirs(composite_dir, exist_ok=True)

def generate_all_composite_images(df):
    skipped_objects = [] 
    for _, row in df.iterrows():
        print(f"\nProcessing {row['correct_object']}...")

        obj1 = row["correct_object"]
        obj2 = row["smaller_object"]
        obj3 = row["larger_object"]

        obj1_mask = row["mask"]
        obj2_row = df[df["correct_object"] == obj2]
        obj3_row = df[df["correct_object"] == obj3]
        #"""
        if (
            not isinstance(obj1_mask, np.ndarray) or
            obj2_row.empty or not isinstance(obj2_row.iloc[0]["mask"], np.ndarray) or
            obj3_row.empty or not isinstance(obj3_row.iloc[0]["mask"], np.ndarray)
        ):
            print(f"Skipping {obj1} due to missing or invalid mask data.")
            skipped_objects.append(obj1)
            continue
        #"""
        obj2_mask = obj2_row.iloc[0]["mask"]
        obj3_mask = obj3_row.iloc[0]["mask"]

        obj1_mask = Image.fromarray((obj1_mask * 255).astype(np.uint8))
        obj2_mask = Image.fromarray((obj2_mask * 255).astype(np.uint8))
        obj3_mask = Image.fromarray((obj3_mask * 255).astype(np.uint8))

        obj1_size = row["total_size"]
        obj2_size = row["smaller_size"]
        obj3_size = row["larger_size"]

        def make_case(mask1, mask2, size1, size2, name1, name2, force_larger_is_first=False):
            if force_larger_is_first:
                size1, size2 = max(size1, size2), min(size1, size2)

            if random.choice([True, False]):
                left_mask, right_mask = mask1, mask2
                left_size, right_size = size1, size2
                left_name, right_name = name1, name2
            else:
                left_mask, right_mask = mask2, mask1
                left_size, right_size = size2, size1
                left_name, right_name = name2, name1

            description = f"{name1} (larger) vs {name2} (smaller)"
            return (left_mask, right_mask, left_size, right_size, description, left_name, right_name)

        def crop_to_mask(mask):
            arr = np.array(mask)
            rows = np.any(arr > 0, axis=1)
            cols = np.any(arr > 0, axis=0)
            if not rows.any() or not cols.any():
                return mask, (0, 0, mask.width, mask.height)
            y_min, y_max = np.where(rows)[0][[0, -1]]
            x_min, x_max = np.where(cols)[0][[0, -1]]
            return mask.crop((x_min, y_min, x_max + 1, y_max + 1)), (x_min, y_min, x_max + 1, y_max + 1)

        def get_real_image(name):
            row_match = df[df["correct_object"] == name]
            if not row_match.empty:
                return Image.open(row_match.iloc[0]["image_path"]).convert("RGB")
            return None

        cases = [
            make_case(obj1_mask, obj2_mask, obj1_size, obj2_size, obj1, obj2),
            make_case(obj2_mask, obj1_mask, obj2_size, obj1_size, obj2, obj1, force_larger_is_first=True),
            make_case(obj3_mask, obj1_mask, obj3_size, obj1_size, obj3, obj1),
            make_case(obj1_mask, obj3_mask, obj1_size, obj3_size, obj1, obj3, force_larger_is_first=True),
        ]

        for left_mask, right_mask, left_size, right_size, description, left_name, right_name in cases:
            print(f"\nGenerating: {description}")

            LARGER_DIM = (250, 250)
            SMALLER_DIM = (80, 80)

            larger_label = description.split("vs")[0].strip()
            left_is_larger = left_name in larger_label

            left_dim = LARGER_DIM if left_is_larger else SMALLER_DIM
            right_dim = SMALLER_DIM if left_is_larger else LARGER_DIM

            # Crop masks and get crop boxes
            left_cropped_mask, left_crop_box = crop_to_mask(left_mask)
            right_cropped_mask, right_crop_box = crop_to_mask(right_mask)

            # Resize masks
            left_resized_mask = left_cropped_mask.resize(left_dim, Image.LANCZOS)
            right_resized_mask = right_cropped_mask.resize(right_dim, Image.LANCZOS)

            # Prepare canvas
            spacing = 20
            canvas_width = left_resized_mask.width + right_resized_mask.width + spacing
            canvas_height = max(left_resized_mask.height, right_resized_mask.height) + 50

            line_y = canvas_height - 50

            def find_mask_bottom(mask_img):
                mask_arr = np.array(mask_img.convert("L"))
                rows = np.where(mask_arr > 0)[0]
                return rows.max() if len(rows) > 0 else mask_img.height - 1

            left_bottom = find_mask_bottom(left_resized_mask)
            right_bottom = find_mask_bottom(right_resized_mask)

            left_x = 0
            left_y = line_y - left_bottom
            right_x = left_resized_mask.width + spacing
            right_y = line_y - right_bottom

            # --- MASK CANVAS ---
            mask_canvas = Image.new("RGB", (canvas_width, canvas_height), "white")
            draw = ImageDraw.Draw(mask_canvas)
            for x in range(0, canvas_width, 20):
                draw.line([(x, line_y), (x + 10, line_y)], fill="black", width=2)

            mask_canvas.paste(left_resized_mask.convert("RGB"), (left_x, left_y))
            mask_canvas.paste(right_resized_mask.convert("RGB"), (right_x, right_y))
            display(mask_canvas)

            # --- REAL IMAGE CANVAS ---
            print("Showing real images...")

            def get_real_image(name):
                row_match = df[df["correct_object"] == name]
                if not row_match.empty:
                    return Image.open(row_match.iloc[0]["image_path"]).convert("RGB")
                return None

            left_real = get_real_image(left_name)
            right_real = get_real_image(right_name)
            
            original_left_mask = left_mask  # before cropping
            original_right_mask = right_mask
            
            if left_real:
                if left_real.size != original_left_mask.size:
                    left_real = left_real.resize(original_left_mask.size, Image.LANCZOS)
                left_real_cropped = left_real.crop(left_crop_box)
                left_real_resized = left_real_cropped.resize(left_resized_mask.size, Image.LANCZOS)
                left_composite = Image.composite(left_real_resized, Image.new("RGB", left_real_resized.size, "white"), left_resized_mask.convert("L"))
            else:
                left_composite = Image.new("RGB", left_resized_mask.size, "gray")
            
            if right_real:
                if right_real.size != original_right_mask.size:
                    right_real = right_real.resize(original_right_mask.size, Image.LANCZOS)
                right_real_cropped = right_real.crop(right_crop_box)
                right_real_resized = right_real_cropped.resize(right_resized_mask.size, Image.LANCZOS)
                right_composite = Image.composite(right_real_resized, Image.new("RGB", right_real_resized.size, "white"), right_resized_mask.convert("L"))
            else:
                right_composite = Image.new("RGB", right_resized_mask.size, "gray")

            real_canvas = Image.new("RGB", (canvas_width, canvas_height), "white")
            real_canvas.paste(left_composite, (left_x, left_y))
            real_canvas.paste(right_composite, (right_x, right_y))
            draw = ImageDraw.Draw(real_canvas)
            for x in range(0, canvas_width, 20):
                draw.line([(x, line_y), (x + 10, line_y)], fill="black", width=2)
            display(real_canvas)

            larger_obj = left_name if left_size > right_size else right_name
            smaller_obj = right_name if left_size > right_size else left_name
            
            # Clean filename
            filename = f"{larger_obj}_larger_than_{smaller_obj}".replace(" ", "_") + "_with_line.png"
            save_path = os.path.join(composite_dir, filename)
            
            # Save real image composite
            real_canvas.save(save_path)
            print(f"Saved to: {save_path}")
    return skipped_objects

skipped_objects = generate_all_composite_images(df)


In [ ]:
skipped_objects

In [ ]:
df =df[~df["correct_object"].isin(skipped_objects)]

In [ ]:
import pandas as pd

# Create a list to store the new rows
new_rows = []

# Iterate through the existing dataframe
for _, row in df.iterrows():
    correct_object = row["correct_object"]
    smaller_object = row["smaller_object"]
    larger_object = row["larger_object"]

    # Generate expected paths for the composite images
    path_to_clean_1 = f"composites/{correct_object}_larger_than_{smaller_object}_with_line.png"
    path_to_counterfact_1 = f"composites/{smaller_object}_larger_than_{correct_object}_with_line.png"
    path_to_clean_2 = f"composites/{larger_object}_larger_than_{correct_object}_with_line.png"
    path_to_counterfact_2 = f"composites/{correct_object}_larger_than_{larger_object}_with_line.png"

    # First row: correct_object vs. smaller_object
    new_rows.append({
        "correct_object": correct_object,
        "comparison_object": smaller_object,
        "total_size": row["total_size"],
        "comparison_size": row["smaller_size"],
        #"attribute": row["attribute"],
        "correct_answer": row["correct_answer"],
        "path_to_clean": path_to_clean_1,
        "path_to_counterfact": path_to_counterfact_1,
    })

    # Second row: correct_object vs. larger_object
    new_rows.append({
        "correct_object": correct_object,
        "comparison_object": larger_object,
        "total_size": row["total_size"],
        "comparison_size": row["larger_size"],
        #"attribute": row["attribute"],
        "correct_answer": row["correct_answer"],
        "path_to_clean": path_to_clean_2,
        "path_to_counterfact": path_to_counterfact_2,
    })

# Convert to a new DataFrame
new_df = pd.DataFrame(new_rows)
new_df

In [ ]:
new_df = new_df[new_df["correct_object"] != "toilet"]
new_df = new_df.dropna()

In [ ]:
# Create new columns for clean_answer and counterfact_answer based on size comparison
new_df["clean_answer"] = new_df.apply(
    lambda row: row["correct_object"] if row["total_size"] > row["comparison_size"] else row["comparison_object"], axis=1
)

new_df["counterfact_answer"] = new_df.apply(
    lambda row: row["comparison_object"] if row["total_size"] > row["comparison_size"] else row["correct_object"], axis=1
)

new_df

In [ ]:
new_df = new_df.drop_duplicates(subset=["clean_answer", "counterfact_answer"], keep="first")


In [ ]:
from PIL import Image
from IPython.display import display

for _, row in new_df.iterrows():
    clean_img = Image.open(row["path_to_clean"])#.convert("RGB")
    counterfact_img = Image.open(row["path_to_counterfact"])#.convert("RGB")

    print("Clean Image:")
    display(clean_img)

    print("Counterfactual Image:")
    display(counterfact_img)


In [ ]:
new_df.to_csv("images_composite_images_with_line.csv", index=False)